In [ ]:
# 完全沒有裝主題的版本，加入confidence_threshold和詳細解釋程式碼功能的註解

import cv2
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
from ultralytics import YOLO

class WebcamApp:

    # 初始化函數，用於建立GUI應用程式
    def __init__(self, window, window_title, model_file, labels_file, confidence_threshold=0.5):
        #設定視窗的基本屬性
        self.window = window
        self.window.title(window_title)
        
        # 設定視訊來源和 YOLO 模型
        self.video_source = 0  # 視訊來源，內建攝像頭(0)
        self.vid = cv2.VideoCapture(self.video_source)  # 使用OpenCV打開視訊捕捉
        
        self.model = YOLO(model_file)  # 載入YOLO模型
        self.labels = self.load_labels(labels_file)  # 載入物件類別標籤
        self.confidence_threshold = confidence_threshold  # 儲存信心閾值
        

        
        # 建立標籤用於顯示偵測結果
        self.label_title = tk.Label(window, text="", font=("Helvetica", 16))
        self.label_title.pack(anchor=tk.N, pady=20)  # 使用anchor屬性設定對齊到頂部(North)
        
        # 建立畫布用於顯示視訊畫面
        self.canvas = tk.Canvas(window, width=480, height=360)
        self.canvas.pack(pady=15)
        
        # 建立描述標籤用於顯示物品描述
        self.label_description = tk.Label(window, text="", font=("Helvetica", 12), wraplength=300, anchor="w", justify="left")
        self.label_description.pack(pady=10)
        
#         # 建立開啟和關閉視訊的按鈕
#         self.btn_open = ttk.Button(window, text="Open Webcam", command=self.open_webcam, state="disabled")
#         self.btn_open.pack(padx=10, pady=5)
        
#         self.btn_close = ttk.Button(window, text="Close Webcam", command=self.close_webcam)
#         self.btn_close.pack(padx=10, pady=5)
        
        # 更新視訊畫面和視窗事件迴圈 - 啟動視窗的事件迴圈，以便不斷更新視訊畫面
        self.update()
        self.window.mainloop()

# # 開啟視訊
#     def open_webcam(self):
#         self.vid = cv2.VideoCapture(self.video_source)
#         self.btn_open["state"] = "disabled"
#         self.btn_close["state"] = "normal"

# # 關閉視訊
#     def close_webcam(self):
#         if self.vid.isOpened():
#             self.vid.release()
#         self.canvas.delete("all")
#         self.btn_open["state"] = "normal"
#         self.btn_close["state"] = "disabled"

    # 更新視訊畫面
    def update(self):
        # 讀取視訊畫面的一個影格
        ret, frame = self.vid.read()

        if ret:
            # 使用 YOLO 模型預測畫面中的物件
            results = self.model.predict(source=frame)

            # 在影格上繪製偵測到的物件框和標籤
            frame_with_rects = self.add_labels(frame, results[0].boxes.xyxy.tolist(), results[0].boxes.cls.tolist())

            # 將畫面轉換成RGB格式（因為OpenCV使用BGR，而PIL使用RGB）
            frame_rgb = cv2.cvtColor(frame_with_rects, cv2.COLOR_BGR2RGB)

            # 在畫面上再次繪製偵測到的物件框和標籤
            frame_with_labels = self.add_labels(frame_rgb, results[0].boxes.xyxy.tolist(), results[0].boxes.cls.tolist())

            # 取得偵測到的物件的類別標籤
            detected_objects = [self.labels[int(c)] for c in results[0].boxes.cls.tolist()]

            # 遍歷每個偵測到的物件，逐一顯示其翻譯和描述
            if detected_objects:
                obj = detected_objects[0]  # 假設只顯示第一個物件的信息

                translation, color = self.get_translation(obj)
                self.label_title.config(text=translation, fg="#%02x%02x%02x" % color)  # 設定文字顏色
                description = self.get_description(obj)  # 取得物件的描述信息

                # 更新GUI中的標籤和描述
                self.label_title.config(text=translation)
                self.label_description.config(text=description)

            # 顯示視訊畫面在GUI中的畫布上
            self.photo = ImageTk.PhotoImage(image=Image.fromarray(frame_with_labels))
            self.canvas.create_image(0, 0, image=self.photo, anchor=tk.NW)

        # 設定每隔10毫秒調用一次自身，以持續更新視訊畫面
        self.window.after(10, self.update)

    def get_translation(self, label):
        translation_dict = {
            "Rivopam": "可那平錠",
            "nexium": "耐適恩錠",
            "Tonec": "痛停錠",
            "BOGEN-50 S.C. TABLETS": "培元糖衣錠",
            "E-WEGEN": "明大育胃源錠",
            "vitamincold": "長安維他命感冒膠囊",
            "tonpicon_capsules_\"S.D.\"": "世達通鼻康膠囊",
            "panadol": "普拿疼"
        }

        # 設定不同翻譯文字的顏色
        color_dict = {
            "Rivopam": (0, 255, 0),
            "nexium": (0, 0, 255),
            "Tonec": (255, 0, 0),
            "BOGEN-50 S.C. TABLETS": (192, 192, 0),
            "E-WEGEN": (255, 0, 255),
            "vitamincold": (0, 255, 255),
            "tonpicon_capsules_\"S.D.\"": (128, 0, 0),
            "panadol": (0, 128, 0)
        }

        translation = translation_dict.get(label, label)  # 如果找不到對應，就返回原英文名稱
        color = color_dict.get(label, (0, 0, 0))  # 如果找不到對應的顏色，就使用黑色

        # 返回包含文字和顏色設定的元組
        return translation, color
    
    def get_description(self, label):
        description_dict = {
            "Rivopam": "藥名: Rivopam 可那平錠\n\n外觀: 膚色圓形藥錠\n\n適應症: 包括抗痙攣(抗癲癇)作用、鎮靜作用、肌肉鬆弛作用及焦慮緩解作用。\n\n用量: 成人20 mg / day，孩童0.2 mg / kg / day\n\n副作用: 欲睡、頭暈或頭昏眼花、倦怠或肌肉無力、口乾、腸胃不適、食慾改變、腹瀉、記憶力受損（健忘）等副作用。",
            "nexium": "藥名: Nexium 耐適恩錠\n\n外觀: 深膚色橢圓形藥錠\n\n適應症: 胃食道逆流性疾病、糜爛性逆流性食道炎、與適當之抗菌劑療法併用治療根除幽門螺旋桿菌、由幽門螺旋桿菌引發之十二指腸潰瘍、NSAID治療相關之胃潰瘍的治療、(ZES)之治療、預防消化性潰瘍再出血之治療。\n\n用量: 成人20-40 mg / day / 1-2次，孩童2.5 mg  / day\n\n副作用: 頭痛、腹脹、腹痛、便祕、腹瀉、噁心。",
            "Tonec": "藥名: Tonec 痛停錠\n\n外觀: 白色圓形藥錠\n\n適應症: 治療退化性關節炎、類風濕性關節炎、僵直性脊椎炎所引起之疼痛及發炎症狀。\n\n用量: 成人100 mg / day / 2次\n\n副作用: 胃腸障礙，如消化不良腹痛、噁心、腹瀉、皮膚紅疹、搔癢、頭暈。",
            "BOGEN-50 S.C. TABLETS": "藥名: BOGEN-50 S.C. TABLETS 培元糖衣錠\n\n外觀: 黃色圓形糖衣錠\n\n適應症: 腳氣病、多發性神經炎、其他維他命B1、B2缺乏症、營養補給\n\n副作用: 尿液呈現黃色",
            "E-WEGEN": "藥名: E-WEGEN 明大育胃源錠\n\n外觀: 綠色圓形藥錠\n\n適應症: 胃、十二指腸潰瘍、胃酸過多症、胃炎、胃痙攣、胃痛。\n\n用量: 成人1-2錠 / day / 3次\n\n副作用: 便祕、噁心、嘔吐、腹瀉。",
            "vitamincold": "藥名: Vitamincold 長安維他命感冒膠囊\n\n外觀: 一半暗紅一半黃色膠囊\n\n適應症: 感冒諸症狀（流鼻水、鼻塞、打噴嚏、咽喉痛、咳嗽、喀痰、發熱、頭痛）之緩解。\n\n用量: 成人1錠 / day / 3次\n\n副作用: 皮膚發疹、發紅。噁心、嘔吐、食慾不振。頭暈、耳鳴。喉嚨疼痛、心跳加速、排尿困難、視覺模糊。",
            "tonpicon_capsules_\"S.D.\"": "藥名: Tonpicon_capsules_\"S.D.\" 世達通鼻康膠囊\n\n外觀: 一半白色一半藍色膠囊\n\n適應症: 緩解過敏性鼻炎、枯草熱所引起之相關症狀(鼻炎、流鼻水、打噴嚏)。减少鼻及咽喉之分泌，鬆弛支氣管使呼吸通暢。\n\n用量: 成人1錠 / day / 4次，孩童1/4錠  / day\n\n副作用: 倦怠感、興奮、口乾、視覺上之困擾。",
            "panadol": "藥名: Panadol 普拿疼\n\n外觀: 一半黃色一半白色橢圓形藥錠\n\n適應症: 緩解感冒之各種症狀(鼻塞、咽喉痛、咳嗽、畏寒、發燒、頭痛、肌肉痠痛)\n\n用量: 成人1錠 / day / 4次，孩童請洽醫師\n\n副作用: 皮膚發疹、發紅。 噁心、嘔吐、食慾不振。頭暈、耳鳴。喉嚨疼痛、心跳加速、排尿困難、視覺模糊。"
        }
        return description_dict.get(label, "暫無介紹")
        

    def add_labels(self, image, rectangles, cls):
        # 定義每個物件類別的顏色映射，有8種物件類別
        class_colors = {
            0: (0, 255, 0),   # 物件類別0的顏色-綠色
            1: (0, 0, 255),   # 紅色
            2: (255, 0, 0),   # 藍色
            3: (192, 192, 0), # 黃色
            4: (255, 0, 255), # 紫色
            5: (0, 255, 255), # 青色
            6: (128, 0, 0),   # 深紅色
            7: (0, 128, 0)    # 深綠色
        }

        for rect, c in zip(rectangles, cls):
            c = int(c)  # 轉換成整數
            color = class_colors.get(c, (0, 0, 0))  # 根據物件類別取得對應的顏色，未知類別使用黑色

            x1, y1, x2, y2 = list(map(int, rect))
            cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)

            label = self.labels[c]  # 根據類別索引取得標籤
            cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        return image


    def load_labels(self, labels_file):
        with open(labels_file, 'r') as f:
            labels = f.read().splitlines()
        return labels

if __name__ == "__main__":
    model_file = 'runs/train/exp/weights/best.pt'
    labels_file = 'classes.txt'
    
    root = tk.Tk()
    root.geometry("428x900")  # 設定視窗大小
    app = WebcamApp(root, "Webcam App", model_file, labels_file)
